In [1]:
!wget https://github.com/karpathy/minbpe/blob/master/tests/taylorswift.txt

--2024-07-29 10:34:00--  https://github.com/karpathy/minbpe/blob/master/tests/taylorswift.txt
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘taylorswift.txt’

taylorswift.txt         [ <=>                ] 998.65K  --.-KB/s    in 0.04s   

2024-07-29 10:34:00 (22.8 MB/s) - ‘taylorswift.txt’ saved [1022614]



In [2]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Pythonic way to iterate consecutive elements
                                          # ids[1:] --> creates a new list that is the original list ids starting from the second element
                                          # zip(ids, ids[1:]) --> pairs each element in ids with its subsequent element, forming a tuple.
                                              # this loop iterates over each of these pairs
        counts[pair] = counts.get(pair, 0) + 1    # returns the valye for the key "pair" from dictionary "counts" // if not already a key, returns 0 instead (default value) // then increments count of "pair" by 1
    return counts

In [9]:
def merge(ids, pair, idx): # Defines a function named merge that takes three parameters:
                                      # ids (a list of integers), pair (a tuple of two integers to search for in ids), and idx (a new integer that will replace occurrences of pair).
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  i = 0
  while i < len(ids):
    # if we are not at the very last position AND the pair matches, replace it
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1

  return newids

# print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))

# tokens2 = merge(tokens, top_pair, 256)

In [18]:
text = ""
with open('taylorswift.txt', 'r', encoding='utf-8') as f:
  text = f.read()
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print('---')
print(tokens[0:75])
print("length:", len(tokens))
print("max:"   , max(tokens))

---
[10, 10, 10, 10, 10, 10, 60, 33, 68, 79, 67, 84, 89, 80, 69, 32, 104, 116, 109, 108, 62, 10, 60, 104, 116, 109, 108, 10, 32, 32, 108, 97, 110, 103, 61, 34, 101, 110, 34, 10, 32, 32, 10, 32, 32, 100, 97, 116, 97, 45, 99, 111, 108, 111, 114, 45, 109, 111, 100, 101, 61, 34, 97, 117, 116, 111, 34, 32, 100, 97, 116, 97, 45, 108, 105]
length: 1022614
max: 226


In [11]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 10000 # the target final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list // preservation for comparison

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)           # function retrieves dictionary with pairs of consecutive tokens and their frequencies
  pair = max(stats, key=stats.get) # stats.get fetches value assosciated with each key in stats // this helps to determine key with highest value
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)      # replaces all occurences of "pair" in "ids" with new tokens "idx"
  merges[pair] = idx               # records merge operation in "mergs" dictiionary, mapping from pair of tokens to the new token index

merging (61, 34) into a new token 256
merging (105, 110) into a new token 257
merging (105, 118) into a new token 258
merging (101, 114) into a new token 259
merging (34, 32) into a new token 260
merging (101, 45) into a new token 261
merging (111, 110) into a new token 262
merging (100, 258) into a new token 263
merging (114, 101) into a new token 264
merging (97, 116) into a new token 265
merging (101, 32) into a new token 266
merging (115, 116) into a new token 267
merging (99, 116) into a new token 268
merging (62, 60) into a new token 269
merging (99, 111) into a new token 270
merging (32, 32) into a new token 271
merging (116, 101) into a new token 272
merging (44, 32) into a new token 273
merging (108, 257) into a new token 274
merging (111, 114) into a new token 275


In [12]:
vocab = {idx: bytes([idx]) for idx in range(256)} # initislises dictionary where key key is an interger randing from 0 to 255 // and corresponding value is a byte array containing that integer.
                                                        # this essentially maps each single-byte value to its byte representation. Which is useful for decoding operations becuase it directly translates
                                                        # byte values into their respective byte representations.
for (p0, p1), idx in merges.items(): # iterates through merges dict which contains mappings of pairs of newly merged token indices
    vocab[idx] = vocab[p0] + vocab[p1] # idx is correct. Value was correctly assigned as value during for loop at first!

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids) # b"" means byte string // this concatenates all the byte array from vocab into a single byte array

  text = tokens.decode("utf-8", errors="replace") # Convert Bytes to String: tokens.decode("utf-8", errors="replace") converts the byte array into a string using UTF-8 encoding.
                                                          # The errors='replace' parameter ensures that if any bytes cannot be decoded properly, they are replaced with a placeholder character
                                                          # (typically �) instead of raising an error.
  return text

print(decode([128]))

�


In [13]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2: # continues process as long as there's at least 2 tokens in the list.
    stats = get_stats(tokens) # previously defined // calculate the frequency of each consecutive pair of tokens in the list
    pair = min(stats, key=lambda p: merges.get(p, float("inf"))) # purpose of line is to find pair of tokens "p" that is least preferable for merging based on whether are they defined in "merges" dictionary
                                                                 # if p is not found, basically will return infinity as default value, effectively prioritising it as least desirable. Because infinite value will always
                                                                 # be greater than any index.

                                                                # further notes:
                                                                      # refer to above. but i generally get the consensus. Due to how lambda function works here, p is all pair chars in stats being iterated over.
                                                                      # when p is not in merges, infinity value is set making it impossible to be ever chosen
                                                                      # on the other hand, if not, then the value inside merges or the merge pair vocab will be returned. The lowest value will always be chosen first
                                                                      # which due to how most frequent values are always added first, this means that more frequent values are iterated first etc.

                                                                      # REFER TO NOTES ABOVE
    if pair not in merges:
      break # nothing else can be merged // exits loop when
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

In [14]:
decode(encode("hello world"))

'hello world'

In [25]:
print({k: v.decode('utf-8') for k, v in vocab.items() if v[0] <= 0x7F})


{0: '\x00', 1: '\x01', 2: '\x02', 3: '\x03', 4: '\x04', 5: '\x05', 6: '\x06', 7: '\x07', 8: '\x08', 9: '\t', 10: '\n', 11: '\x0b', 12: '\x0c', 13: '\r', 14: '\x0e', 15: '\x0f', 16: '\x10', 17: '\x11', 18: '\x12', 19: '\x13', 20: '\x14', 21: '\x15', 22: '\x16', 23: '\x17', 24: '\x18', 25: '\x19', 26: '\x1a', 27: '\x1b', 28: '\x1c', 29: '\x1d', 30: '\x1e', 31: '\x1f', 32: ' ', 33: '!', 34: '"', 35: '#', 36: '$', 37: '%', 38: '&', 39: "'", 40: '(', 41: ')', 42: '*', 43: '+', 44: ',', 45: '-', 46: '.', 47: '/', 48: '0', 49: '1', 50: '2', 51: '3', 52: '4', 53: '5', 54: '6', 55: '7', 56: '8', 57: '9', 58: ':', 59: ';', 60: '<', 61: '=', 62: '>', 63: '?', 64: '@', 65: 'A', 66: 'B', 67: 'C', 68: 'D', 69: 'E', 70: 'F', 71: 'G', 72: 'H', 73: 'I', 74: 'J', 75: 'K', 76: 'L', 77: 'M', 78: 'N', 79: 'O', 80: 'P', 81: 'Q', 82: 'R', 83: 'S', 84: 'T', 85: 'U', 86: 'V', 87: 'W', 88: 'X', 89: 'Y', 90: 'Z', 91: '[', 92: '\\', 93: ']', 94: '^', 95: '_', 96: '`', 97: 'a', 98: 'b', 99: 'c', 100: 'd', 101: 'e'